In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**fixed seed setting**

In [2]:
import torch
import random
import numpy as np
import os

seed = 50
os.environ['PYTHONHASHSEED']=str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

**device setting**

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

**Data Loader setting**

In [4]:
import pandas as pd

data_path = '/kaggle/input/plant-pathology-2020-fgvc7/'

train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [5]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(train,
                               test_size=0.1,
                                stratify=train[['healthy', 'multiple_diseases', 'rust', 'scab']],
                               random_state=50)

In [6]:
import cv2
from torch.utils.data import Dataset
import numpy as np

class ImageDataset(Dataset):
    def __init__(self, df, img_dir, transform=None, is_test=False):
        super().__init__()
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_id = self.df.iloc[idx, 0]
        img_path = self.img_dir + img_id + '.jpg'
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform is not None:
            image = self.transform(image=image)['image']
        if self.is_test:
            return image
        else:
            label = np.argmax(self.df.iloc[idx, 1:5])
            return image, label

In [7]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

transform_train = A.Compose([
    A.Resize(450, 650),
    A.RandomBrightnessContrast(brightness_limit=0.2,contrast_limit=0.2, p=0.3),
    A.VerticalFlip(p=0.2),
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(
        shift_limit=0.1,
        scale_limit=0.2,
        rotate_limit=30,
        p=0.3
    ),
    A.OneOf([A.Emboss(p=1),
            A.Sharpen(p=1),
            A.Blur(p=1)], p=0.3),
    A.PiecewiseAffine(p=0.3),
    A.Normalize(),
    ToTensorV2()
])

/opt/conda/lib/python3.10/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.17). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
/opt/conda/lib/python3.10/site-packages/albumentations/core/validation.py:45: UserWarning: This augmenter is very slow. Try to use ``ElasticTransformation`` instead, which is at least 10x faster.
  original_init(self, **validated_kwargs)


In [8]:
transform_test = A.Compose([
    A.Resize(450, 650),
    A.Normalize(),
    ToTensorV2()
])

In [9]:
img_dir = '/kaggle/input/plant-pathology-2020-fgvc7/images/'

dataset_train = ImageDataset(train, img_dir=img_dir, transform=transform_train)
dataset_valid = ImageDataset(valid, img_dir=img_dir, transform=transform_test)

**Multi Processing setting**

In [10]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

In [11]:
from torch.utils.data import DataLoader

batch_size = 4

loader_train = DataLoader(dataset_train, batch_size=batch_size,
                         shuffle=True, worker_init_fn=seed_worker,
                         generator=g, num_workers=2)
loader_valid = DataLoader(dataset_valid, batch_size=batch_size,
                         shuffle=False, worker_init_fn=seed_worker,
                         generator=g, num_workers=2)

**Transfer Learning EfficientNet**

In [12]:
!pip install efficientnet-pytorch==0.7.1

In [13]:
from efficientnet_pytorch import EfficientNet

# pretrained num_classes 1000 -> transfer learning num_classes 4

# method 1 : define num_classes
model = EfficientNet.from_pretrained('efficientnet-b7', num_classes=4)

model.to(device)

# method 2 : modify fc layer output
"""
model = EfficientNet.from_pretrained('efficientnet-b7')
model._fc = nn.Sequential(
    nn.Linear(model._fc.in_features, fc.out_features),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(model._fc.out_features=4)
)
"""

Loaded pretrained weights for efficientnet-b7


"\nmodel = EfficientNet.from_pretrained('efficientnet-b7')\nmodel._fc = nn.Sequential(\n    nn.Linear(model._fc.in_features, fc.out_features),\n    nn.ReLU(),\n    nn.Dropout(p=0.5),\n    nn.Linear(model._fc.out_features=4)\n)\n"

In [14]:
import torch.nn as nn

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00006, weight_decay=0.0001)

In [15]:
from sklearn.metrics import roc_auc_score
from tqdm.notebook import tqdm
epochs = 5

# train loop
for epoch in range(epochs):
    
    model.train()
    epoch_train_loss = 0
    
    for images, labels in tqdm(loader_train):

        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, labels)
        epoch_train_loss += loss.item()
        loss.backward()
        optimizer.step()

    print(f'epoch [{epoch+1}/{epochs}] train_loss : {epoch_train_loss/len(loader_train):.4f}')
    
    # validation loop
    model.eval()
    epoch_valid_loss = 0
    pred_lists = []
    true_onehot_list = []
        
    with torch.no_grad():
        for images, labels in loader_valid:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            epoch_valid_loss += loss.item()

            preds = torch.softmax(outputs.cpu(), dim=1).numpy()
            true_onehot = torch.eye(4, device=device)[labels].cpu().numpy() 
            
            pred_lists.extend(preds)
            true_onehot_list.extend(true_onehot)

    print(f'epoch [{epoch+1}/{epochs}] epoch_valid_loss : {epoch_valid_loss/len(loader_valid):.4f} / ROC AUC : {roc_auc_score(true_onehot_list, pred_lists):.4f}')

  0%|          | 0/410 [00:00<?, ?it/s]

epoch [1/5] train_loss : 0.6942
epoch [1/5] epoch_valid_loss : 0.2704 / ROC AUC : 0.9405


  0%|          | 0/410 [00:00<?, ?it/s]

epoch [2/5] train_loss : 0.3597
epoch [2/5] epoch_valid_loss : 0.2525 / ROC AUC : 0.9647


  0%|          | 0/410 [00:00<?, ?it/s]

epoch [3/5] train_loss : 0.2544
epoch [3/5] epoch_valid_loss : 0.1722 / ROC AUC : 0.9694


  0%|          | 0/410 [00:00<?, ?it/s]

epoch [4/5] train_loss : 0.1842
epoch [4/5] epoch_valid_loss : 0.4817 / ROC AUC : 0.9618


  0%|          | 0/410 [00:00<?, ?it/s]

epoch [5/5] train_loss : 0.1393
epoch [5/5] epoch_valid_loss : 0.1665 / ROC AUC : 0.9753


In [16]:
dataset_test = ImageDataset(test, img_dir=img_dir,
                           transform=transform_test, is_test=True)
loader_test = DataLoader(dataset_test, batch_size=batch_size,
                        shuffle=False, worker_init_fn=seed_worker,
                        generator=g, num_workers=2)

In [17]:
model.eval()

preds = np.zeros((len(test), 4))

with torch.no_grad():
    for i, images in enumerate(loader_test):
        images = images.to(device)
        outputs = model(images)

        preds_part = torch.softmax(outputs.cpu(), dim=1).squeeze().numpy()
        preds[i*batch_size:(i+1)*batch_size] += preds_part

In [18]:
submission[['healthy', 'mutiple_diseases', 'rust', 'scab']] = preds
submission.to_csv('submission.csv', index=False)